<a href="https://colab.research.google.com/github/Dr-Carlos-Villasenor/PatternRecognition/blob/main/PR10_02_Autoresgresor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Reconocimiento de Patrones
##Dr. Carlos Villaseñor
## Autoregresor AR(p)


# Carga de datos

Paso 1. Cargar las siguientes bibliotecas

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf, pacf

Paso 2. Cargue los datos

In [ ]:
!wget "https://raw.githubusercontent.com/Dr-Carlos-Villasenor/PatternRecognition/main/Dataset/daily-min-temperatures.csv"
data = pd.read_csv('daily-min-temperatures.csv')
print(data.head())

In [5]:
x = np.asanyarray(data[['Temp']])

# ACF y PACF

Paso 3. Clacula las gráficas ACF y PACF

In [ ]:
ACF = acf(x)
plot_acf(ACF)

In [ ]:
PACF = pacf(x)
plot_pacf(PACF)

# Particionar en el tiempo

In [ ]:
train_percentage=0.75
slice_point = int(data.shape[0]*train_percentage)
train = data.Temp[:slice_point]
test =  data.Temp[slice_point:]
plt.plot(np.arange(0,slice_point), train, label='Train set')
plt.plot(np.arange(slice_point,data.shape[0]), test, label='Test set')
plt.legend()

# Tabular datos

In [15]:
def sliding_window(train, delays=5):
  data2 = pd.DataFrame(train)
  for i in range(1, delays + 1):
      data2 = pd.concat([data2, train.shift(-i)], axis=1)
  data2 = data2[:-delays]
  x = np.asanyarray(data2.iloc[:,0:-1])
  y = np.asanyarray(data2.iloc[:,-1])
  return x, y

In [16]:
from sklearn.model_selection import train_test_split

delays = 5
x, y = sliding_window(train, delays=delays)
xtrain, xval, ytrain, yval = train_test_split(x, y)

# Crear modelo lineal


In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(xtrain, ytrain)

print('Train score: ', model.score(xtrain, ytrain))
print('Validation score: ', model.score(xval, yval))

# Simular prediciones del sistema

In [ ]:
# Simulate prediction with pretrained model
input = np.zeros((1,delays))
pred = []
for i in range(data.shape[0]):
  pred.append(model.predict(input))
  input[0, 0:delays-1] =  input[0, 1:delays]
  input[0,-1] = data.Temp[i]


# Draw all prediction and data
plt.figure(figsize=(27,9))
plt.plot(np.arange(0, slice_point), train, '.k', linewidth=1, label='Train set')
plt.plot(np.arange(slice_point,data.shape[0]), test,'.b', linewidth=1, label='Test set')
plt.plot(pred, '-r', linewidth=1, label='Prediction')
plt.title('Temperature one-step predictor')
plt.legend()



In [ ]:
xtest, ytest =sliding_window(test, delays=delays)
print('Test score: ', model.score(x, y))